<a href="https://colab.research.google.com/github/davidvlaminck/AWVGeneric/blob/master/UseCases/AddKenmerkOnAssettype/AddKenmerkOnAssettype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pas eerst onderstaande parameter aan en voer het script uit. Een cookie vervalt binnen de 24 uur. Je kan [hier](https://sites.google.com/mow.vlaanderen.be/em-infra/afspraken-conventies/awv-cookie-ophalen) lezen hoe je de AWV ACM cookie kan ophalen.

In [1]:
awv_acm_cookie = '8245a6a9e3b14cf59fede5f3cf8a13c3'

Voer onderstaande script uit om de GitHub repo te clonen.

In [2]:
import os, shutil
if os.path.exists('AWVGeneric'):
  shutil.rmtree('AWVGeneric')
! git clone https://github.com/davidvlaminck/AWVGeneric

Cloning into 'AWVGeneric'...
remote: Enumerating objects: 775, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 775 (delta 181), reused 183 (delta 140), pack-reused 533 (from 1)
Receiving objects: 100% (775/775), 935.44 KiB | 6.15 MiB/s, done.
Resolving deltas: 100% (466/466), done.


Voer onderstaande script uit om alle assets van een bepaald type te downloaden. Je kan de typeUri parameter aanpassen, zie https://apps.mow.vlaanderen.be/eminfra/admin/installatietypes

In [6]:
import sys
sys.path.insert(0, '/content/AWVGeneric')  # adds the Python files from the GitHub repo

import json
from pathlib import Path

from API.EMInfraClient import EMInfraClient
from API.Enums import Environment, AuthType


eminfra_client = EMInfraClient(env=Environment.PRD, auth_type=AuthType.COOKIE, cookie=awv_acm_cookie)

gemigreerd_naar_kenmerk = eminfra_client.get_kenmerktype_by_naam('Gemigreerd naar')
gemigreerd_van_kenmerk = eminfra_client.get_kenmerktype_by_naam('Gemigreerd van')

uri_types_add_gemigreerd_naar = [
    'https://lgc.data.wegenenverkeer.be/ns/installatie#ZoutBijlaadPlaats',
    'https://lgc.data.wegenenverkeer.be/ns/installatie#Zoutsilo',
    'https://lgc.data.wegenenverkeer.be/ns/installatie#Pekeltank',
    'https://lgc.data.wegenenverkeer.be/ns/installatie#MotorLegacy',
    'https://lgc.data.wegenenverkeer.be/ns/installatie#Fietstel',
    'https://lgc.data.wegenenverkeer.be/ns/installatie#FietsTelSysteemLegacy',
    'https://lgc.data.wegenenverkeer.be/ns/installatie#FietsTelDisplayLegacy']
uri_types_add_gemigreerd_van = [
    'https://wegenenverkeer.data.vlaanderen.be/ns/onderdeel#Datakabel',
    'https://wegenenverkeer.data.vlaanderen.be/ns/installatie#Zoutbijlaadplaats',
    'https://wegenenverkeer.data.vlaanderen.be/ns/onderdeel#Silo',
    'https://wegenenverkeer.data.vlaanderen.be/ns/onderdeel#Tank',
    'https://wegenenverkeer.data.vlaanderen.be/ns/onderdeel#Elektromotor',
    'https://wegenenverkeer.data.vlaanderen.be/ns/installatie#Fietstelinstallatie',
    'https://wegenenverkeer.data.vlaanderen.be/ns/onderdeel#Fietstelsysteem',
    'https://wegenenverkeer.data.vlaanderen.be/ns/onderdeel#FietstelDisplay']

assettypes = list(eminfra_client.get_all_assettypes())

for uri_type in uri_types_add_gemigreerd_naar:
    assettype = next((x for x in assettypes if x.uri == uri_type), None)
    if assettype is None:
        print(f'Assettype {uri_type} niet gevonden')
        continue

    bestaande_kenmerken = eminfra_client.get_kenmerken_by_assettype_uuid(assettype.uuid)
    gemigreerd_naar = next((x for x in bestaande_kenmerken
                            if x.kenmerkType.uuid == gemigreerd_naar_kenmerk.uuid), None)
    if gemigreerd_naar is not None:
        print(f'Kenmerk {gemigreerd_naar_kenmerk.naam} reeds aanwezig op {assettype.uri}')
        continue

    eminfra_client.add_kenmerk_to_assettype(assettype_uuid=assettype.uuid,
                                            kenmerktype_uuid=gemigreerd_naar_kenmerk.uuid)
    print(f'Kenmerk {gemigreerd_naar_kenmerk.naam} toegevoegd aan {assettype.uri}')

for uri_type in uri_types_add_gemigreerd_van:
    assettype = next((x for x in assettypes if x.uri == uri_type), None)
    if assettype is None:
        print(f'Assettype {uri_type} niet gevonden')
        continue

    bestaande_kenmerken = eminfra_client.get_kenmerken_by_assettype_uuid(assettype.uuid)
    gemigreerd_van = next((x for x in bestaande_kenmerken
                           if x.kenmerkType.uuid == gemigreerd_van_kenmerk.uuid), None)
    if gemigreerd_van is not None:
        print(f'Kenmerk {gemigreerd_van_kenmerk.naam} reeds aanwezig op {assettype.uri}')
        continue

    eminfra_client.add_kenmerk_to_assettype(assettype_uuid=assettype.uuid,
                                            kenmerktype_uuid=gemigreerd_van_kenmerk.uuid)
    print(f'Kenmerk {gemigreerd_van_kenmerk.naam} toegevoegd aan {assettype.uri}')

Kenmerk Gemigreerd naar toegevoegd aan https://lgc.data.wegenenverkeer.be/ns/installatie#ZoutBijlaadPlaats
Kenmerk Gemigreerd naar toegevoegd aan https://lgc.data.wegenenverkeer.be/ns/installatie#Zoutsilo
Kenmerk Gemigreerd naar toegevoegd aan https://lgc.data.wegenenverkeer.be/ns/installatie#Pekeltank
Kenmerk Gemigreerd naar toegevoegd aan https://lgc.data.wegenenverkeer.be/ns/installatie#MotorLegacy
Kenmerk Gemigreerd naar toegevoegd aan https://lgc.data.wegenenverkeer.be/ns/installatie#Fietstel
Kenmerk Gemigreerd naar toegevoegd aan https://lgc.data.wegenenverkeer.be/ns/installatie#FietsTelSysteemLegacy
Kenmerk Gemigreerd naar toegevoegd aan https://lgc.data.wegenenverkeer.be/ns/installatie#FietsTelDisplayLegacy
Kenmerk Gemigreerd van toegevoegd aan https://wegenenverkeer.data.vlaanderen.be/ns/onderdeel#Datakabel
Kenmerk Gemigreerd van toegevoegd aan https://wegenenverkeer.data.vlaanderen.be/ns/installatie#Zoutbijlaadplaats
Kenmerk Gemigreerd van toegevoegd aan https://wegenenverkee